# <center>  Deep Neural Network (DNN) using plain TensorFlow

## <font color = 'blue'> Get the data

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from collections import defaultdict
from time import time

%store -r
%store

Stored variables and their in-db values:
X_16_val                  -> array([[-0.05248989,  0.95262376, -0.95081096, ...
X_32_val                  -> array([[-0.26412427,  0.80742202,  0.90784197, ...
X_32test_std              -> defaultdict(<class 'list'>, {0: array([[-0.2641242
X_32train_std             -> array([[-0.71199092,  0.15545522, -0.78672279, ...
X_test                    -> defaultdict(<class 'list'>, {0: array([[[ 0.003087
X_test_std                -> defaultdict(<class 'list'>, {0: array([[ -5.248988
X_train                   -> array([[[ -8.82267195e-04,   4.11425252e-03,  -7.3
X_train_std               -> array([[-0.71199092,  0.15545522, -0.78672279, ...
snrs                      -> [-20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 
y_16_val                  -> array([7, 6, 5, ..., 1, 4, 7])
y_32_test                 -> defaultdict(<class 'list'>, {0: array([4, 6, 6, ..
y_32_train                -> array([4, 2, 0, ..., 1, 4, 6])
y_32_val                  -> array([4, 

In [2]:
print("Training data: ", X_32train_std.shape, "and labels: ", y_32_train.shape)
print(" ")
print("Test data:")
print("Total", len(X_32test_std), X_32test_std[18].shape, "arrays for SNR values:")
print(sorted(X_32test_std.keys()))

Training data:  (80000, 32) and labels:  (80000,)
 
Test data:
Total 20 (4000, 32) arrays for SNR values:
[-20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


In [3]:
# tf.reset_default_graph()

## <font color = 'blue'> Design and train the DNN

In [4]:
# ----------------- Initialize parameters -----------------------


n_hidden1 = 1024
n_hidden2 = 1024
n_outputs = 8

X = tf.placeholder(tf.float32, shape=(None, 32))

y = tf.placeholder(tf.int64, shape=(None))

# ------------------- Define layers -----------------------

from tensorflow.contrib.layers import fully_connected

def layer(X, n_neurons, activation):
    layer = fully_connected(X, n_neurons, activation_fn = activation, 
                            weights_initializer = tf.contrib.layers.xavier_initializer())
    dropout_layer = tf.layers.dropout(layer, rate = 0.5, training = True)
    return dropout_layer

layer1 = layer(X, n_hidden1, activation = tf.nn.relu)

layer2 = layer(layer1, n_hidden2, activation = tf.nn.relu)

logits = fully_connected(layer2, n_outputs, activation_fn = None, weights_initializer = tf.contrib.layers.xavier_initializer())

# ----------------- Specify performance measure ----------------------

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)

optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        
# ------------------ Execution phase ----------------------------------    

n_epochs = 1000
batch_size = 1024
n_train = X_train_std.shape[0]
n_iter = n_train//batch_size

saver = tf.train.Saver()

min_loss = np.infty
epochs_without_improvement = 0 
max_epochs_without_improvement = 20   

acc_test = defaultdict(list)

start = time()
path = "./DNNdropout5.ckpt"

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for epoch in range(n_epochs):
        for iteration in range(n_iter):
            rand_indices = np.random.choice(n_train,batch_size) #select random samples to form mini batches   
            X_batch, y_batch = X_32train_std[rand_indices], y_32_train[rand_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        valid_loss, acc_val = sess.run([loss, accuracy], feed_dict={X: X_32_val, y: y_32_val})
        
        # Early stopping 
        
        if valid_loss < min_loss:
            save_path = saver.save(sess, path)
            min_loss = valid_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement > max_epochs_without_improvement:
                print("** EARLY STOPPING ** ")
                break
        print("{}\tValidation loss: {:.6f}\tMinimum loss: {:.6f}\tAccuracy on validation set: {:.2f}".format(
            epoch, valid_loss, min_loss, acc_val))

print(" ")
with tf.Session() as sess:
    saver.restore(sess, path)
    for snr in snrs:
        acc_test[snr] = accuracy.eval(feed_dict={X: X_32test_std[snr], y: y_32_test[snr]})

print("Training and testing took %f minutes"%(float(time() - start)/60))

0	Validation loss: 1.455948	Minimum loss: 1.455948	Accuracy on validation set: 0.40
1	Validation loss: 1.359515	Minimum loss: 1.359515	Accuracy on validation set: 0.44
2	Validation loss: 1.332091	Minimum loss: 1.332091	Accuracy on validation set: 0.45
3	Validation loss: 1.312313	Minimum loss: 1.312313	Accuracy on validation set: 0.46
4	Validation loss: 1.302476	Minimum loss: 1.302476	Accuracy on validation set: 0.46
5	Validation loss: 1.297685	Minimum loss: 1.297685	Accuracy on validation set: 0.46
6	Validation loss: 1.285098	Minimum loss: 1.285098	Accuracy on validation set: 0.47
7	Validation loss: 1.267716	Minimum loss: 1.267716	Accuracy on validation set: 0.48
8	Validation loss: 1.249361	Minimum loss: 1.249361	Accuracy on validation set: 0.49
9	Validation loss: 1.236627	Minimum loss: 1.236627	Accuracy on validation set: 0.49
10	Validation loss: 1.224271	Minimum loss: 1.224271	Accuracy on validation set: 0.49
11	Validation loss: 1.217040	Minimum loss: 1.217040	Accuracy on validation 

## <font color = 'blue'> Test the DNN

In [5]:
for snr in snrs:
    print("DNN's test accuracy on %d dB SNR samples = "%(snr), acc_test[snr])   

DNN's test accuracy on -20 dB SNR samples =  0.12575
DNN's test accuracy on -18 dB SNR samples =  0.12325
DNN's test accuracy on -16 dB SNR samples =  0.13325
DNN's test accuracy on -14 dB SNR samples =  0.126
DNN's test accuracy on -12 dB SNR samples =  0.149
DNN's test accuracy on -10 dB SNR samples =  0.17725
DNN's test accuracy on -8 dB SNR samples =  0.29375
DNN's test accuracy on -6 dB SNR samples =  0.3665
DNN's test accuracy on -4 dB SNR samples =  0.4095
DNN's test accuracy on -2 dB SNR samples =  0.47375
DNN's test accuracy on 0 dB SNR samples =  0.559
DNN's test accuracy on 2 dB SNR samples =  0.68475
DNN's test accuracy on 4 dB SNR samples =  0.80475
DNN's test accuracy on 6 dB SNR samples =  0.8245
DNN's test accuracy on 8 dB SNR samples =  0.82075
DNN's test accuracy on 10 dB SNR samples =  0.831
DNN's test accuracy on 12 dB SNR samples =  0.832
DNN's test accuracy on 14 dB SNR samples =  0.822
DNN's test accuracy on 16 dB SNR samples =  0.826
DNN's test accuracy on 18 dB

## <font color = 'blue'> Visualize DNN's performance

In [6]:
import matplotlib.pyplot as plt

plt.style.use('classic')

fig = plt.figure(figsize=(8, 4), dpi=100)
x = snrs
y = list(acc_test.values())
plt.plot(x, y, marker="o", linewidth=2.0, linestyle='dashed', color='royalblue')
plt.axis([-20, 20, 0, 1])
plt.xticks(np.arange(min(x), max(x)+1, 2.0))
plt.yticks(np.arange(0, 1, 0.10))

ttl = plt.title('SNR vs Accuracy', fontsize=16)
ttl.set_weight('bold')
plt.xlabel('SNR (dB)', fontsize=14)
plt.ylabel('Test accuracy', fontsize=14)
plt.grid()

plt.show()

In [7]:
acc_test

defaultdict(list,
            {-20: 0.12575001,
             -18: 0.12325,
             -16: 0.13325,
             -14: 0.126,
             -12: 0.149,
             -10: 0.17725,
             -8: 0.29374999,
             -6: 0.36649999,
             -4: 0.4095,
             -2: 0.47375,
             0: 0.55900002,
             2: 0.68475002,
             4: 0.80475003,
             6: 0.82450002,
             8: 0.82075,
             10: 0.83099997,
             12: 0.83200002,
             14: 0.82200003,
             16: 0.82599998,
             18: 0.83450001})